In [2]:
# Import libraries
import arcpy
import matplotlib.pyplot as plt
import numpy as np
from arcpy import env
import pandas as pd

In [ ]:
# Identify variables in the project
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("AFFH Automation Testing")[0]
layout = aprx.listLayouts("AFFH Automation Testing")[0]

#end the file path with two slashes "\\"
file_path = "J:\GIS\GIS_Projects\CDBG_DR\Projects_2025\DR_25001_HCFCD_DR_MIT_AFFH\Pdf\\"

In [4]:
# Define layer names
layers = {
    "centroid_layer": "Project Centroid",
    "project_area_layer": "Project Area",
    "block_group_layer": "Project Benefit Block Groups",
    "benefit_area_layer": "Project Benefit Area",
    "buffer_layer": "Project Centroid Buffer (1 Mile)",
    "age_65_layer": "Percent Age 65 Plus",
    "crime_layer": "Crime Index",
    "disability_layer": "Percent of Households with Disability",
    "minority_concentration_layer": "Percent Other than White Not Hispanic",
    "poverty_layer": "Percent of Persons in Poverty",
    "substandard_housing_layer": "Substandard Housing",
    "large_household_layer": "Large Households"
}

In [5]:
# Define data sources
data_sources = {
    "age_65_data": "Data Sources: Harris County Flood Control, US Census Bureau ACS5 2015-2019, Harris County Housing & Community Development (HCHCD)",
    "crime_data": "Data Sources: Harris County Flood Control, US Census Bureau ACS5 2015-2019, ESRI Total Crime Index (AGS) 2022, Harris County Housing & Community Development (HCHCD)",
    "disability_data": "Data Sources: Harris County Flood Control, US Census Bureau ACS5 2015-2019, Harris County Housing & Community Development (HCHCD)",
    "minority_concentration_data": "Data Sources: Harris County Flood Control, US Census Bureau ACS5 2015-2019, Harris County Housing & Community Development (HCHCD)",
    "poverty_data": "Data Sources: Harris County Flood Control, US Census Bureau ACS5 2015-2019, Harris County Housing & Community Development (HCHCD)",
    "substandard_housing_data": "Data Sources: Harris County Flood Control, US Census Bureau, HUD Comprehensive Housing Affordable Strategy (CHAS) 2016-2020, Harris County Housing & Community Development (HCHCD)",
    "large_household_data": "Data Sources: Harris County Flood Control, US Census Bureau ACS5 2015-2019, HUD, Harris County Housing & Community Development (HCHCD)"
}

In [ ]:
# Apply symbology to layers
symbology_layers = {
    "project_area_layer": "Project Area Symbology",
    "centroid_layer": "Benefit Area Centroids Symbology",
    "block_group_layer": "Benefit Block Group Symbology",
    "benefit_area_layer": "Benefit Area Symbology",
    "buffer_layer": "Buffer Symbology"
}

for layer_key, symbology_name in symbology_layers.items():
    arcpy.management.ApplySymbologyFromLayer(
        [layer for layer in m.listLayers() if layer.name == layers[layer_key]][0],
        [layer for layer in m.listLayers() if layer.name == symbology_name][0]
    )

In [6]:
# Clear definition queries and labels for specific layers
layer_names = [
    layers["centroid_layer"], layers["project_area_layer"], 
    layers["block_group_layer"], layers["benefit_area_layer"],
    layers["buffer_layer"]
]

for lyr in m.listLayers():
    if lyr.name in layer_names:
        if lyr.supports("DEFINITIONQUERY"):
            lyr.definitionQuery = ""
        if lyr.supports("SHOWLABELS"):
            lyr.showLabels = False

In [7]:
# Access and filter project data
centroid_layer = layers["benefit_area_layer"]
layer = [lyr for lyr in m.listLayers() if lyr.name == centroid_layer][0]

data = []
with arcpy.da.SearchCursor(layer, ["ProjectID", "Project_Description"]) as cursor:
    for row in cursor:
        data.append(row)

df = pd.DataFrame(data, columns=["ProjectID", "Project_Description"])
unique_projects_df = df.drop_duplicates(subset="ProjectID")

pd.set_option('display.max_colwidth', None)
print(unique_projects_df)

          ProjectID                                                                      Project_Description
0   P518-14-00-E001                                                    Hahl North Stormwater Detention Basin
1   P518-08-02-E001             Channel Conveyance Improvements and Stormwater Detention Basin on P118-08-00
2   P118-00-00-E008                         Halls Bayou Channel Conveyance Improvements Downstream of Hopper
3   N100-00-00-E004                                          Cloverleaf Area Drainage Improvements - Phase 2
4   C147-00-00-E002                                         Channel Conveyance Improvements along C147-00-00
5   C118-00-00-E001                                    Channel Conveyance Improvements along Saltwater Ditch
6   B509-04-00-E002                                            Red Bluff Regional Stormwater Detention Basin
7   U501-06-00-E002        South Mayde Creek Channel Conveyance Improvements and Stormwater Detention Basins
8   G103-00-00-E003

In [8]:
#Select your project ID from the list above
project_id = "U501-07-00-E002"

In [9]:
# Apply a definition query to layers
query = f"ProjectID = '{project_id}'"
for layer_name in layer_names:
    layer = m.listLayers(layer_name)[0]
    layer.definitionQuery = query

In [14]:
# Get and set the map frame extent
extent_layer = m.listLayers(layers["block_group_layer"])[0]
mf = layout.listElements("MAPFRAME_ELEMENT", "Main Map")[0]
extent = mf.getLayerExtent(extent_layer, False, True)
buffer_factor = .05
buffered_extent = arcpy.Extent(
    extent.XMin - extent.width * buffer_factor,
    extent.YMin - extent.height * buffer_factor,
    extent.XMax + extent.width * buffer_factor,
    extent.YMax + extent.height * buffer_factor
)
mf.camera.setExtent(buffered_extent)

In [15]:
# Function to update text elements
def update_text_element(title, data_source, layer_visibility):
    text_element = layout.listElements("TEXT_ELEMENT", "title")[0]
    text_element.text = title
    text_element = layout.listElements("TEXT_ELEMENT", "Data Source")[0]
    text_element.text = data_source
    for layer_name, visible in layer_visibility.items():
        layer = [lyr for lyr in m.listLayers() if lyr.name == layer_name]
        if layer:
            layer[0].visible = visible

# Age 65 and Older Map

In [25]:
# Get the project description
with arcpy.da.SearchCursor(centroid_layer, ["Project_Description"]) as cursor:
    project_description = next(cursor)[0]  # Get the project name from the "Project_Description" field

# Create the text string
text_string = f"{project_description} - Age 65 and Older"

# Update the title text element
layout.listElements("TEXT_ELEMENT", "title")[0].text = text_string

# Example: Update for the Age 65 and Older Map
text_string = f"{project_description} - Age 65 and Older"
update_text_element(text_string, data_sources["age_65_data"], {
    layers["age_65_layer"]: True,
    layers["crime_layer"]: False,
    layers["disability_layer"]: False,
    layers["minority_concentration_layer"]: False,
    layers["poverty_layer"]: False,
    layers["substandard_housing_layer"]: False,
    layers["large_household_layer"]: False
})
# Export the layout to PDF
layout.exportToPDF(f"{file_path}{text_string}.pdf", resolution=300, image_quality="BEST", embed_fonts=True, image_compression="ADAPTIVE")

'J:\\GIS\\GIS_Projects\\CDBG_DR\\Projects_2024\\DR_AFFH_MIT_Phase2_24007\\Pdf\\Stormwater Detention Basin on South Mayde Creek near the Grand Parkway - Phase 1 - Age 65 and Older.pdf'

# Crime

In [26]:
# Get the project description
with arcpy.da.SearchCursor(centroid_layer, ["Project_Description"]) as cursor:
    project_description = next(cursor)[0]  # Get the project name from the "Project_Description" field

# Create the text string
text_string = f"{project_description} - Crime"

# Update the title text element
layout.listElements("TEXT_ELEMENT", "title")[0].text = text_string

# Crime Map
text_string = f"{project_description} - Crime"
update_text_element(text_string, data_sources["crime_data"], {
    layers["age_65_layer"]: False,
    layers["crime_layer"]: True,
    layers["disability_layer"]: False,
    layers["minority_concentration_layer"]: False,
    layers["poverty_layer"]: False,
    layers["substandard_housing_layer"]: False,
    layers["large_household_layer"]: False
})
layout.exportToPDF(f"{file_path}{text_string}.pdf", resolution=300, image_quality="BEST", embed_fonts=True, image_compression="ADAPTIVE")

'J:\\GIS\\GIS_Projects\\CDBG_DR\\Projects_2024\\DR_AFFH_MIT_Phase2_24007\\Pdf\\Stormwater Detention Basin on South Mayde Creek near the Grand Parkway - Phase 1 - Crime.pdf'

# Household Disability

In [27]:
# Get the project description
with arcpy.da.SearchCursor(centroid_layer, ["Project_Description"]) as cursor:
    project_description = next(cursor)[0]  # Get the project name from the "Project_Description" field

# Create the text string
text_string = f"{project_description} - Household Disability"

# Update the title text element
layout.listElements("TEXT_ELEMENT", "title")[0].text = text_string

# Household Disability Map
text_string = f"{project_description} - Household Disability"
update_text_element(text_string, data_sources["disability_data"], {
    layers["age_65_layer"]: False,
    layers["crime_layer"]: False,
    layers["disability_layer"]: True,
    layers["minority_concentration_layer"]: False,
    layers["poverty_layer"]: False,
    layers["substandard_housing_layer"]: False,
    layers["large_household_layer"]: False
})
layout.exportToPDF(f"{file_path}{text_string}.pdf", resolution=300, image_quality="BEST", embed_fonts=True, image_compression="ADAPTIVE")

'J:\\GIS\\GIS_Projects\\CDBG_DR\\Projects_2024\\DR_AFFH_MIT_Phase2_24007\\Pdf\\Stormwater Detention Basin on South Mayde Creek near the Grand Parkway - Phase 1 - Household Disability.pdf'

# Non-White or Hispanic Population Concentration

In [28]:
# Get the project description
with arcpy.da.SearchCursor(centroid_layer, ["Project_Description"]) as cursor:
    project_description = next(cursor)[0]  # Get the project name from the "Project_Description" field

# Create the text string
text_string = f"{project_description} - Minority Concentration"

# Update the title text element
layout.listElements("TEXT_ELEMENT", "title")[0].text = text_string

# Export map with minority concentration
text_string = f"{project_description} - Minority Concentration"
update_text_element(text_string, data_sources["minority_concentration_data"], {
    layers["age_65_layer"]: False,
    layers["crime_layer"]: False,
    layers["disability_layer"]: False,
    layers["minority_concentration_layer"]: True,
    layers["poverty_layer"]: False,
    layers["substandard_housing_layer"]: False,
    layers["large_household_layer"]: False
})
layout.exportToPDF(f"{file_path}{text_string}.pdf", resolution=300, image_quality="BEST", embed_fonts=True, image_compression="ADAPTIVE")

'J:\\GIS\\GIS_Projects\\CDBG_DR\\Projects_2024\\DR_AFFH_MIT_Phase2_24007\\Pdf\\Stormwater Detention Basin on South Mayde Creek near the Grand Parkway - Phase 1 - Minority Concentration.pdf'

# Percent in Poverty

In [29]:
# Get the project description
with arcpy.da.SearchCursor(centroid_layer, ["Project_Description"]) as cursor:
    project_description = next(cursor)[0]  # Get the project name from the "Project_Description" field

# Create the text string
text_string = f"{project_description} - Poverty"

# Update the title text element
layout.listElements("TEXT_ELEMENT", "title")[0].text = text_string

# Export map with Poverty layer
text_string = f"{project_description} - Poverty"
update_text_element(text_string, data_sources["poverty_data"], {
    layers["age_65_layer"]: False,
    layers["crime_layer"]: False,
    layers["disability_layer"]: False,
    layers["minority_concentration_layer"]: False,
    layers["poverty_layer"]: True,
    layers["substandard_housing_layer"]: False,
    layers["large_household_layer"]: False
})
layout.exportToPDF(f"{file_path}{text_string}.pdf", resolution=300, image_quality="BEST", embed_fonts=True, image_compression="ADAPTIVE")

'J:\\GIS\\GIS_Projects\\CDBG_DR\\Projects_2024\\DR_AFFH_MIT_Phase2_24007\\Pdf\\Stormwater Detention Basin on South Mayde Creek near the Grand Parkway - Phase 1 - Poverty.pdf'

# Substandard Housing

In [30]:
# Get the project description for title update
with arcpy.da.SearchCursor(centroid_layer, ["Project_Description"]) as cursor:
    project_description = next(cursor)[0]  # Get the project name from the "Project_Description" field

# Create the text string
text_string = f"{project_description} - Substandard Housing"

# Update the title text element
layout.listElements("TEXT_ELEMENT", "title")[0].text = text_string

# Export map with Substandard Housing layer
text_string = f"{project_description} - Substandard Housing"
update_text_element(text_string, data_sources["substandard_housing_data"], {
    layers["age_65_layer"]: False,
    layers["crime_layer"]: False,
    layers["disability_layer"]: False,
    layers["minority_concentration_layer"]: False,
    layers["poverty_layer"]: False,
    layers["substandard_housing_layer"]: True,
    layers["large_household_layer"]: False
})
layout.exportToPDF(f"{file_path}{text_string}.pdf", resolution=300, image_quality="BEST", embed_fonts=True, image_compression="ADAPTIVE")

'J:\\GIS\\GIS_Projects\\CDBG_DR\\Projects_2024\\DR_AFFH_MIT_Phase2_24007\\Pdf\\Stormwater Detention Basin on South Mayde Creek near the Grand Parkway - Phase 1 - Substandard Housing.pdf'

# Large Households

In [31]:
# Get the project description
with arcpy.da.SearchCursor(centroid_layer, ["Project_Description"]) as cursor:
    project_description = next(cursor)[0]  # Get the project name from the "Project_Description" field

# Create the text string
text_string = f"{project_description} - Large Households"

# Update the title text element
layout.listElements("TEXT_ELEMENT", "title")[0].text = text_string

# Export map with Large Households layer
text_string = f"{project_description} - Large Households"
update_text_element(text_string, data_sources["large_household_data"], {
    layers["age_65_layer"]: False,
    layers["crime_layer"]: False,
    layers["disability_layer"]: False,
    layers["minority_concentration_layer"]: False,
    layers["poverty_layer"]: False,
    layers["substandard_housing_layer"]: False,
    layers["large_household_layer"]: True
})
layout.exportToPDF(f"{file_path}{text_string}.pdf", resolution=300, image_quality="BEST", embed_fonts=True, image_compression="ADAPTIVE")

'J:\\GIS\\GIS_Projects\\CDBG_DR\\Projects_2024\\DR_AFFH_MIT_Phase2_24007\\Pdf\\Stormwater Detention Basin on South Mayde Creek near the Grand Parkway - Phase 1 - Large Households.pdf'